In [66]:
import pandas as pd
import re
import itertools
import sqlite3
import json

In [21]:
# Define import functions
def import_and_clean(path):
    file = pd.read_json(path)
    data = pd.DataFrame(list(itertools.chain(*file['message'].apply(lambda x: x['items']))))
    data['earliest_pub_year'] = data['published'].apply(lambda x: x['date-parts'][0][0])
    data['cleaned_abstract'] = data['abstract'].apply(lambda x: re.sub("<.*?>|\n|Abstract|abstract", "", str(x)))
    return data

In [22]:
climate_articles_2013 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2013.json')
climate_articles_2014 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2014.json')
climate_articles_2015 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2015.json')
climate_articles_2016 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2016.json')
climate_articles_2017 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2017.json')
climate_articles_2018 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2018.json')
climate_articles_2019 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2019.json')
climate_articles_2020 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2020.json')
climate_articles_2021 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2021.json')
climate_articles_2022 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2022.json')
climate_articles_2023 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2023.json')

In [23]:
climate_articles = pd.concat([
    climate_articles_2013,
    climate_articles_2014,
    climate_articles_2015,
    climate_articles_2016,
    climate_articles_2017,
    climate_articles_2018,
    climate_articles_2019,
    climate_articles_2020,
    climate_articles_2021,
    climate_articles_2022,
    climate_articles_2023
])

In [ ]:
climate_articles.shape
# 284,231 articles extracted- still need to clean/ check for potential DOI duplicates

(284231, 56)

In [53]:
climate_articles_unique = climate_articles.drop_duplicates(subset = 'DOI', keep ='first')

In [54]:
climate_articles_unique.shape
# all were unique DOI
# 284,231

(284231, 56)

In [55]:
climate_articles_unique_english = climate_articles_unique[climate_articles_unique['language'] == 'en']
climate_articles_unique_english.shape
# 224,958 articles published in English, but note some may be missing abstracts at this point

(224958, 56)

In [61]:
final_df = climate_articles_unique_english.reset_index(drop = True)

In [62]:
final_df.to_json('climate_articles_unique_english.json')

In [72]:
for column in final_df.columns:
    if final_df[column].apply(lambda x: isinstance(x, (dict, list))).any():
        final_df[column] = final_df[column].apply(json.dumps)

In [73]:
final_df.head()

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,...,funder,issue-title,article-number,updated-by,accepted,earliest_pub_year,cleaned_abstract,chair,translator,clinical-trial-number
0,"{""date-parts"": [[2024, 8, 22]], ""date-time"": ""...",29,Springer Science and Business Media LLC,1,"[{""start"": {""date-parts"": [[2013, 11, 24]], ""d...","{""domain"": [], ""crossmark-restriction"": false}","[""Nature Clim Change""]","{""date-parts"": [[2014, 1]]}",10.1038/nclimate2060,journal-article,...,NaN,NaN,NaN,NaN,NaN,2013,nan,NaN,NaN,NaN
1,"{""date-parts"": [[2024, 9, 10]], ""date-time"": ""...",28,American Meteorological Society,18,NaN,"{""domain"": [], ""crossmark-restriction"": false}",NaN,"{""date-parts"": [[2013, 9, 15]]}",10.1175/jcli-d-12-00476.1,journal-article,...,NaN,NaN,NaN,NaN,NaN,2013,The ratio of warming to cumulat...,NaN,NaN,NaN
2,"{""date-parts"": [[2024, 6, 15]], ""date-time"": ""...",18,MDPI AG,1,"[{""start"": {""date-parts"": [[2013, 5, 3]], ""dat...","{""domain"": [], ""crossmark-restriction"": false}","[""Climate""]",NaN,10.3390/cli1010004,journal-article,...,NaN,NaN,NaN,NaN,NaN,2013,The rise in global average temperature over th...,NaN,NaN,NaN
3,"{""date-parts"": [[2023, 5, 19]], ""date-time"": ""...",5,Springer Science and Business Media LLC,NaN,"[{""start"": {""date-parts"": [[2013, 8, 1]], ""dat...","{""domain"": [], ""crossmark-restriction"": false}","[""Nature""]",NaN,10.1038/nature.2013.13464,journal-article,...,NaN,NaN,NaN,NaN,NaN,2013,nan,NaN,NaN,NaN
4,"{""date-parts"": [[2024, 9, 15]], ""date-time"": ""...",72,Wiley,3,"[{""start"": {""date-parts"": [[2013, 3, 18]], ""da...","{""domain"": [], ""crossmark-restriction"": false}","[""WIREs Climate Change""]","{""date-parts"": [[2013, 5]]}",10.1002/wcc.211,journal-article,...,NaN,NaN,NaN,NaN,NaN,2013,"In temperate climates, mortality is higher in ...",NaN,NaN,NaN


In [74]:
conn = sqlite3.connect("climate_articles_unique_english.db")
final_df.to_sql("unique_english", conn, if_exists="replace", index=False)
conn.close()